In [1]:
import pickle as pkl
import numpy as np
import wfdb
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [3]:
class Segment:
    def __init__(self, signal=1, label=1, normal_ratio=0):
        self.signal = signal
        self.label = label
        self.normal_ratio = normal_ratio


def create_index_df(desired_segment_len=3600, basic_arr_path="data/mit-bih-arrhythmia-database-1.0.0"):
    desired_segment_len = 3600
    basic_arr_path = "data/mit-bih-arrhythmia-database-1.0.0"
    arr_db = wfdb.get_record_list('mitdb')
    num_samples_in_record = 30 * 60 * 360
    segment_dict = defaultdict(list)
    segment_list = list()
    for _, record_id in enumerate(arr_db):
        record_path = os.path.join(basic_arr_path, str(record_id))

        ann = wfdb.rdann(record_path, 'atr', sampto=num_samples_in_record, return_label_elements=['description'])
        df = pd.DataFrame({'description': ann.description, 'sample': ann.sample})
        counter = 0
        reset_flag = True
        allowed_labels = ['Normal beat']
        normal_counter = 0
        for i in range(1, df.shape[0] - 1):
            curr_label, curr_sample = df.loc[i, ['description', 'sample']]
            if curr_label == 'Normal beat':
                normal_counter += 1
            if reset_flag:
                start_sample = curr_sample
                ann_num_start = i
                normal_counter = 0
                allowed_labels = ['Normal beat']
            next_label, next_sample = df.loc[i + 1, ['description', 'sample']]
            if curr_label == next_label or next_label in allowed_labels or len(allowed_labels) < 2:
                if next_label not in allowed_labels:
                    allowed_labels.append(next_label)
                ann_num_end = i+1
                counter += next_sample - curr_sample
                reset_flag = False
                if counter > desired_segment_len:
                    segment_dict['record_id'].extend([record_id])
                    segment_dict['start_sample'].extend([start_sample])
                    segment_dict['label'].extend([curr_label])
                    counter = 0
                    reset_flag = True
                    signal = wfdb.rdsamp(record_path, sampfrom=start_sample, sampto=start_sample + 3600)[0][:, 0]
                    normal_ratio = normal_counter / (ann_num_end - ann_num_start)
                    segment_list.append(Segment(signal, allowed_labels[-1], normal_ratio))
            else:
                counter = 0
                normal_counter = 0
                reset_flag = True
                allowed_labels = ['Normal beat']

    #     return pd.DataFrame(segment_dict)
    return segment_list

In [4]:
segment_list = create_index_df()
# Next step: radnomly select labeled segments according to Table 1 here:
# "Arrhythmia detection using deep convolutional neural network with long duration ECG signals"
# num_labels_dict = {
#     'Normal beat': 283
#     'Left bundle branch block beat': 103
#                   }

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101


116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117
117


214
214
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
215
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219


In [5]:
len(segment_list)

5149